### Indice
* [ Librarys](#1)
    <br>

* [Dataset](#1)
    <br>
    
* [data_trasnform](#2)

    <br>

* [Study_1:   n_estimators, colsample_bytree](#3)
    <br>
    
    * [Objective Study RandomForest](#1.1)
    * [Best trial]()
    * [Plot_optimization_history](#1.1)
    * [Plot_param_importances]()
    * [plot_parallel_coordinate]()
<br>
<br>
* [Study_2:  eta (learning rate), max_depth, gamma](#3)
    <br>
    
    * [Objective Study RandomForest](#1.1)
    * [Best trial]()
    * [Plot_optimization_history](#1.1)
    * [Plot_param_importances]()
    * [plot_parallel_coordinate]()
<br>
<br>
* [Marge Studys and Save Json](#3)

In [1]:
#Standar Librarys
import pandas as pd
import numpy as np
import scipy
import optuna
from sklearn.metrics import recall_score
from sklearn.metrics import plot_confusion_matrix

# data preparation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

# remove warnings
import warnings
warnings.filterwarnings('ignore')

#utis 
from utils import *

#model Optimizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
#================= data===========================#
df = pd.read_csv('../data/bank-additional-full.csv',sep = ';')
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
X,y = data_trasnform(df, smote=False)

    Study_1 , with the Hyperpameters:  n_estimators, max_depth,max_features

In [12]:
#========================== OPTUNA -- RandomForest ===========================#
def objective(trial,data=X,target=y):
    
     #max_depth, n_estiamtoders, max_feature
    
    # hyperparameter 
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000,100),
        'max_depth': trial.suggest_int('max_depth', 5, 31,2), # o 21
        'max_features': trial.suggest_float('max_features',0.2,0.9)
    }
    
    #==================kfold=====================#
    skf = StratifiedKFold(n_splits=5)
    model = RandomForestClassifier(**params)
    accuracies = []

    for idx in skf.split(X, y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        ytrain = y[train_idx]

        xtest = X[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain,ytrain)
        preds = model.predict(xtest)
        fold_acc = recall_score(ytest, preds) #recall 
        accuracies.append(fold_acc)
        
    return scipy.stats.gmean(accuracies) #media geometrica spipy

In [13]:
study = optuna.create_study(direction='maximize') #maxizer
study.optimize(objective, n_trials=25) #100
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-07 16:06:37,283] A new study created in memory with name: no-name-4d8a0a4e-1081-4e95-b744-809bda221640
[W 2022-11-07 16:06:46,545] Trial 0 failed with parameters: {'n_estimators': 700, 'max_depth': 7, 'max_features': 0.2311929397487059} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\jonat\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\jonat\AppData\Local\Temp/ipykernel_32320/1889234229.py", line 26, in objective
    model.fit(xtrain,ytrain)
  File "C:\Users\jonat\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\jonat\anaconda3\lib\site-packages\joblib\parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "C:\Users\jonat\anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\U

KeyboardInterrupt: 

In [ ]:
study_1 = study.best_trial.params
study_1

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)